# [Python에서 MySQL 실습](https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html)

## 예제1: MySQL 접속 및 조회 

- STEP 1: 모듈 임포트 
    > pip install PyMySQL

In [1]:
# STEP 1
import pymysql 

- STEP 2: MySQL Connection 연결 
    - [RuntimeError 발생 시](https://stackoverflow.com/questions/54477829/cryptography-is-required-for-sha256-password-or-caching-sha2-password)

connect() 함수를 이용하면 MySQL host내 DB와 직접 연결할 수 있습니다.
- user: user name
- passwd: 설정한 패스워드
- host: DB가 존재하는 host
- db: 연결할 데이터베이스 이름
- charset: 인코딩 설정

In [17]:
con = pymysql.connect(host='localhost', user='urstory', password='1234',
                        db='classicmodels', charset='utf8') # 한글처리 (charset = 'utf8')

- STEP 3: Connection으로부터 Cursor 생성 

In [18]:
cur = con.cursor(pymysql.cursors.DictCursor) #dictinary로 표현해줌, 기본값은 튜플

- STEP 4: SQL문 실행 

In [12]:
sql = "show tables;"
cur.execute(sql)

8

In [13]:
# 데이타 Fetch
rows = cur.fetchall()
rows   # 전체 rows

[{'Tables_in_classicmodels': 'customers'},
 {'Tables_in_classicmodels': 'employees'},
 {'Tables_in_classicmodels': 'offices'},
 {'Tables_in_classicmodels': 'orderdetails'},
 {'Tables_in_classicmodels': 'orders'},
 {'Tables_in_classicmodels': 'payments'},
 {'Tables_in_classicmodels': 'productlines'},
 {'Tables_in_classicmodels': 'products'}]

In [10]:
import pandas as pd 

pd.DataFrame(rows)

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


- STEP 5: DB 연결 종료 

In [20]:
if cur is not None:
    cur.close()
if con is not None:
    con.close()

Error: Already closed

## 예제2: 테이블 생성 & 데이터 삽입/변경/삭제 

In [15]:
# STEP 1
import pymysql 

In [21]:
con = pymysql.connect(host='localhost', user='urstory', password='1234',
                        db='classicmodels', charset='utf8') # 한글처리 (charset = 'utf8')

In [22]:
cur = con.cursor(pymysql.cursors.DictCursor)

- 테이블 생성 

In [23]:
sql = '''CREATE TABLE user_test ( 
id int(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, 
name varchar(255), 
address varchar(255) 
) 
''' 

cur.execute(sql)
con.commit()

In [24]:
sql = "desc user_test"
cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,,None,
2,address,varchar(255),YES,,None,


- insert

In [25]:
sql = '''INSERT INTO user_test (name, address) 
    VALUES ('홍길동', '서울');'''

cur.execute(sql)
con.commit()

In [26]:
sql = "select * from user_test"
cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

,id,name,address
0,1,홍길동,서울


In [36]:
insert_data = ['심청', '부산']

sql = '''
INSERT INTO user_test (name, address)
VALUES (%s, %s);'''

cur.execute(sql, insert_data)
con.commit()

In [37]:
sql = "select * from user_test"
cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

,id,name,address
0,2,홍길동,부산
1,3,신사임당,부산
2,4,이름1,경기
3,5,이름2,경기
4,6,이름3,서울
5,7,이름4,서울
6,8,이름5,서울
7,9,심청,부산


- update

In [27]:
sql = '''
UPDATE user_test
SET address = '경기도'
WHERE id = 1;
'''

cur.execute(sql)
con.commit()

In [28]:
sql = "select * from user_test"
cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

,id,name,address
0,1,홍길동,경기도


- delete

In [29]:
sql = '''
DELETE FROM user_test
WHERE id = 1;
'''

cur.execute(sql)
con.commit()

In [30]:
sql = "select * from user_test"
cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

""


## 예제3: Placeholder 사용하기 
> 만약 DB내 데이터에 대해 대량 삽입/변경/삭제가 필요한데, 조건이 다 다르다면? Placeholder를 이용할 수 있습니다!

Placeholder의 특징은,
- 두번째 파라미터에 들어간 데이터 순서대로 SQL이 적용되고,
- 특히 문자의 경우 따옴표 등의 특수문자들이 자동으로 이스케이프(Escape)되어 처리됩니다. (완전 간편!)
- 문자열, 숫자 등에 관계 없이 대치할 값은 모두 `%s`로 쓰입니다. (일반 문자열에서 사용하는 `%s`, `%d`와는 다름)
- `%s`는 컬럼 값을 대치할 때만 사용할 수 있습니다.

### execute(sql, a-data)
하나의 데이터만 적용시키는 경우

In [31]:
a_data = ['홍길동', '부산']

sql = '''
INSERT INTO user_test (name, address) 
VALUES (%s, %s);
'''

cur.execute(sql, a_data)
con.commit()

In [32]:
a_data = ['신사임당', '부산']

sql = '''
INSERT INTO user_test (name, address) 
VALUES (%s, %s);
'''

cur.execute(sql, a_data)
con.commit()

In [33]:
sql = """
select * from user_test where address = %s;
"""

cur.execute(sql, '부산')

rows = cur.fetchall()
pd.DataFrame(rows)

,id,name,address
0,2,홍길동,부산
1,3,신사임당,부산


### executemany(sql, multiple-data)
여러 개의 데이터를 한번에 대량으로 적용시키는 경우 

In [34]:
datas = [
    ['이름1', '경기'],
    ['이름2', '경기'],
    ['이름3', '서울'],
    ['이름4', '서울'],
    ['이름5', '서울'],
]

sql = """
INSERT INTO user_test (name, address) 
VALUES (%s, %s);
"""

cur.executemany(sql, datas)
con.commit()

In [35]:
sql = """
select * from user_test;
"""

cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

,id,name,address
0,2,홍길동,부산
1,3,신사임당,부산
2,4,이름1,경기
3,5,이름2,경기
4,6,이름3,서울
5,7,이름4,서울
6,8,이름5,서울


In [11]:

def solution(my_string, overwrite_string, s):
    # 앞부분 자르기
    front = my_string[:s]
    # 뒷부분 자르기
    back = my_string[s+len(overwrite_string):]
    # 앞부분 + 덮어쓰는 문자열 + 뒷부분을 이어 붙이기
    result = front + overwrite_string + back
    return result

print(solution("He11oWor1d", "lloWorl", 2))
print(solution("Program29b8UYP", "merS123", 7))

HelloWorld
ProgrammerS123
